In [ ]:
import itertools
import copy
import pickle
import numpy as np


                    #####################################################
                    #                Auxiliary Functions                #
                    #####################################################

def is_symmetric(poly):         # Check if polynomial is symmetric in q and t
    R = FractionField(QQ['q','t','x'])  # type: ignore
    q,t,x = R.gens()
    poly1 = R(poly)     # Casting in new ring
    poly2 = poly1
    poly2(q=x)
    poly2(t=q)
    poly2(x=t)
    return (poly == poly2)

def is_increasing_list(lst):
    r"""
        Checks if a list is weakly increasing.
    """
    i = 0
    check = True
    while i < len(lst)-1 and check:
        check = (lst[i]<=lst[i+1])
        i += 1
    return check


                    #####################################################
                    #           Sorted Sandpile Configuration           #
                    #####################################################


class SandpileSortConfig():

    def __init__(self, sandp, conf, permut, sort = True, verts = []):      ## Defines the class SandpileSortConfig
        r"""
            Definition of the class SandpileSortConfig.
            
            The arguments given are a sandpile and a partition of the vertex set (without sink).
        """

        ### TODO: check if the arguments given correspond to a good SandpileSortConfig

        ### Handle the type of configuration given
        if isinstance(conf, SandpileConfig):  #type: ignore
            config = deepcopy({v:conf[v] for v in sandp.nonsink_vertices()})  #type: ignore
        elif isinstance(conf, list):
            vert = sandp.nonsink_vertices()
            config = deepcopy({vert[i]:conf[i] for i in range(len(vert))})    #type: ignore
        elif isinstance(conf, dict):
            config = deepcopy(conf)                                           #type: ignore
        else:
            print(type(conf))
            raise TypeError

        self.sandpile_struct = sandp                                          # Define the sandpile_struct as sandp
        self.perm_group = permut                                              # Define the permutation group on the sandpile
        if verts == []:                                                       # Define the nonsink vertex set
            self.vertices = sandp.nonsink_vertices()
        else:
            self.vertices = verts
        self.sink = self.sandpile_struct.sink()                               # Define the sink vertex (efficiency!)

        if sort:
            # Reorder the configuration based on the permutation group
            result = []
            pos = {}
            index = 0
            for part in self.perm_group:
                for i in part:
                    pos.update({i:index})
                    index += 1
                part_conf = [config[i] for i in part]     # Get the partial configuration associated with an orbit
                part_conf.sort()                          # Sort it
                result = result + part_conf               # Modify the configuration
            sort_conf = {i:result[pos[i]] for i in self.vertices}
        else:
            sort_conf = config

        self.sandpile_config = SandpileConfig(sandp, sort_conf) # type: ignore  # Define the configuration for the sorted sandpile
        

    def __repr__(self):                             ## Returns a description of the class Sandpile2
        return "A sorted configuration for a sandpile with vertices {} and sink {}".format(self.sandpile_struct.vertices(), self.sink)
    

    def sandpile_struct(self):
        r"""
            Returns the underlying sandpile structure.
        """
        return self.sandpile_config.sandpile()

    
    def sort(self, change = True):
        r"""
            Rearrange the configuration values by increasing order on each orbit.

            If change is True, it modifies the configuration, if it is False, it just returns the sorted one.
        """
        result = []
        pos = {}
        index = 0

        for part in self.perm_group:
            for i in part:
                pos.update({i:index})
                index += 1
            part_conf = [self.sandpile_config[i] for i in part]     # Get the partial configuration associated with an orbit
            part_conf.sort()                          # Sort it
            result = result + part_conf               # Modify the configuration
        sort_conf = {i:result[pos[i]] for i in self.vertices}
        if change:
            self.sandpile_config = SandpileConfig(self.sandpile_struct, sort_conf) # type: ignore
        return sort_conf
    

    def __deepcopy__(self, memo):
        r"""
            Overrides the deepcopy method for dict.
        """
        c = SandpileSortConfig(self.sandpile_struct, dict(self.sandpile_config), self.perm_group)
        c.sandpile_config.__dict__.update(self.sandpile_config.__dict__)
        return c


    def __eq__(self, other):                        ## Compares two sorted configurations to see if they are equivalent under the permutation group
        r"""
            Checks if the two sorded configurations are the same or not.
        """
        return self.sandpile_config == other.sandpile_config

    
    def is_recurrent(self):                         ## Check if the configuration is recurrent
        r"""
            Check if the configuration is recurrent.
            
            This function just calls the function is_recurrent() from the class SandpileConfig. 
        """
        return self.sandpile_config.is_recurrent()
    

    def is_stable(self):                            ## Check if the configuration is stable
        r"""
            Check if the configuration is stable.

            This function just calls the function is_stable() from the class SandpileConfig.
        """
        return self.sandpile_config.is_stable()


    def is_sorted(self):                            ## Check if the configuration has already been sorted
        r"""
            Check if the configuration has already been sorted.
        """
        for perm in self.perm_group:
            conf = [self.sandpile_config[i] for i in perm]
        return is_increasing_list(conf)


    def deg(self):                                  ## Returns the degree of the configuration
        r"""
            Returns the degree of the configuration.
        """
        return self.sandpile_config.deg()


    def topple_sink(self, sorting = True):          ## Topples the sink vertex
        r"""
            Topple the sink and change the configuration stored.
        """
        #for vert in self.sandpile_struct.neighbors(self.sandpile_struct.sink()):
        #    self.sandpile_config[vert] += 1
        c = dict(self.sandpile_config)
        for e in self.sandpile_struct.outgoing_edge_iterator(self.sink):
            c[e[1]] += e[2]
        self.sandpile_config = SandpileConfig(self.sandpile_struct, c)    #type: ignore
        if sorting:
            self.sort()
        return self.sandpile_config
    

    def topple_vertex(self, vert, sorting = True):                  ## Topples a vertex of the configuration
        r"""
            Topple a given vertex of the sandpile. This changes the configuration stored.
            
            This function just calls the function fire_vertex() from the class SandpileConfig and applies it to self.sandpile_config. 
        """
        self.sandpile_config = self.sandpile_config.fire_vertex(vert)
        if sorting:
            self.sort()
        return self.sandpile_config
    

    def topple_sequence(self, list, sorting = True):                ## Topples a sequence of vertices
        r"""
            Topple a list of vertices of the sandpile. This changes the configuration stored.
            
            This function calls iteratively the function fire_vertex() from the class SandpileConfig and applies it to self.sandpile_config. 
        """
        for vert in list:
            self.sandpile_config = self.sandpile_config.fire_vertex(vert)
        if sorting:
            self.sort()
        return self.sandpile_config
    

    def single_topple(self, vert, threshold = 0, sorting = True):   ## Toppling sending one grain for each arc
        r"""
            Topple a vertex by sending a grain to each edge.
        """
        neigh = self.sandpile_struct.edges(vert)
        for v in neigh:
            if (v[1] != self.sink) and (v[2] > threshold):
                self.sandpile_config[v[1]] += 1
                self.sandpile_config[vert] -= 1
        if sorting:
            self.sort()
        return self.sandpile_config


    def __invert__(self, sorting = True):                           ## Topplings until stable
        r"""
            Modifies the Sorted Configuration to get the stable configuration.

            The function calls the SandpileConfig.__invert__ function, then applies the sorting.
        """
        self.sandpile_config = ~self.sandpile_config
        if sorting:
            self.sort()
        return self.sandpile_config


    def level(self):                                                ## Returns the level statistic of the configuration
        r"""
            Returns the level of the configuration.
        """
        not_inc_sink = sum([v[2] for v in self.sandpile_struct.to_undirected().edges()]) - self.sandpile_struct.out_degree(self.sink)
        return (- not_inc_sink + self.sandpile_config.deg())
    

    def delay(self, order = [], check_rec = True):                  ## Returns the delay statistic of the configuration
        r"""
            Given a reading order of nonsink vertices, this function computes the configuration's delay.

            - order         : the order for reading vertices. If undefined, the decreasing order on vertices is assumed.
            - check_rec     : this option can be used to override the is_recurrent() call.
        """
        if (not self.is_recurrent()) and check_rec:           # Check if the configuration is recurrent
            print(self.sandpile_config)
            print(self.sandpile_config.is_recurrent())
            self.sandpile_struct.show()
            raise Exception("The sorted configuration is not recurrent, hence delay is not defined.")

        nonsink_vert = self.vertices

        if order == []:                                     # No order has been assigned: take decreasing order
            order = copy.copy(nonsink_vert)
            order.sort()
            order.reverse()
        
        delay = 0                                                   # The delay value
        loop_count = 0                                              # The loop count
        not_toppled = copy.copy(nonsink_vert)                                  # A list with the vertices still to be toppled
        self.topple_sink(sorting = False)                                  # Start by toppling the sink
        while len(not_toppled) > 0:
            for ind in order:                               # Search for non-toppled vertices in the right order
                value = self.sandpile_config[ind]
                if (self.sandpile_struct.out_degree(ind) <= value) and (ind in not_toppled):
                                                            # Can be toppled!
                    delay += loop_count                             # Raise the delay by suitable amount
                    self.topple_vertex(ind, sorting = False)        # Topple the vertex without sorting it back!
                    not_toppled.remove(ind)                         # Remove the vertex from not-toppled
            loop_count += 1
        self.sort()
        return delay
    

    def k_delay(self, order = [], check_rec = True):             ## Returns the NEW delay statistic of the configuration
        r"""
            Given a reading order of nonsink vertices, this function computes the configuration's new conjectured delay.
            - k             : the multeplicity of all edges (except incident to the sink).
            - order         : the order for reading vertices. If undefined, the decreasing order on vertices is assumed.
            - check_rec     : this option can be used to override the is_recurrent() call.
        """
        if (not self.is_recurrent()) and check_rec:           # Check if the configuration is recurrent
            print(self.sandpile_config)
            print(self.sandpile_config.is_recurrent())
            self.sandpile_struct.show()
            raise Exception("The sorted configuration is not recurrent, hence delay is not defined.")

        k = max([self.sandpile_struct.degree(v) for v in self.vertices])

        nonsink_vert = self.vertices
        n = len(nonsink_vert)

        if order == []:                                     # No order has been assigned: take decreasing order
            order = copy.copy(nonsink_vert)
            order.sort()
            order.reverse()

        toppl = [0]*n                                           # Stores information on how many partial topplings are still needed
        finalv = [k]*n                                          # We exit the loop when toppl == finalv
        delay = 0
        plus = 0
        self.topple_sink(sorting = False)                               # Start by toppling the sink
        while toppl != finalv:                               # Until everything has been toppled k times...
            for i in range(len(order)):
                if (self.sandpile_struct.out_degree(order[i]) <= self.sandpile_config[order[i]]) and (toppl[i] == 0):
                                                                        # Can be toppled for the first time!
                    self.single_topple(order[i], threshold = toppl[i], sorting = False)
                    toppl[i] += 1
                    delay += plus
                else:
                    if toppl[i] < finalv[i] and toppl[i] > 0:                   # Topple later time...
                        self.single_topple(order[i], threshold = toppl[i], sorting = False)
                        toppl[i] += 1
            plus += 1
        #print("The configuration {} has k-delay {}".format(self.sandpile_config, delay))
        self.sort()
        return delay


    def show(self, sink = True, colors = False, heights = False, directed = False):     ## Returns a drawing of the configuration
        r"""
            Returns a drawing of the sorted sandpile configuration.
        """
        # Define the color of each part
        palette = rainbow(len(self.perm_group)+1) # type: ignore
        if sink:
            col ={**{palette[0]:[self.sink]},**{palette[i+1]:self.perm_group[i] for i in range(len(self.perm_group))}}
        else:
            col = {palette[i+1]:self.perm_group[i] for i in range(len(self.perm_group))}
        
        # Call SandpileConfig.show()
        self.sandpile_config.show(sink = sink, colors = colors, heights = heights, directed = directed, vertex_colors = col)
    



                    #####################################################
                    #                   Sorted Sandpile                 #
                    #####################################################

class SortedSandpile():

    def __init__(self, graph, sink, permut, opt=[], sort_rec = []):    ## Initialize the class SortedSandpile
        r"""
            Definition of the class SortedSandpile.
            
            The arguments given are a sandpile and a partition of the vertex set (without sink).
        """

        ### TODO: check if the arguments given correspond to a good SandpileSortConfig
        
        self.sandpile_struct = Sandpile(graph, sink) # type: ignore                         # Define the Sandpile structure
        self.vertices = self.sandpile_struct.nonsink_vertices()                             # Define the nonsink vertices
        self.perm_group = permut                                                            # Define the permutation group on the graph vertices
        self.specific_opt = opt                                                             # Define the specific options, to optimize in specific cases
        self.sorted_rec = sort_rec                                                          # Eventually store the sorted recurrent configurations if computed


    def __repr__(self):                                 ## Description of SortedSandpile class
        return "A sorted sandpile on vertices {} and sink {}.".format(self.sandpile_struct.vertices(), self.sandpile_struct.sink())
    

    def _max_stable(self):                              ## Returns maximal stable configuration
        r"""
            Returns the maximal stable sorted configuration of the sandpile.
        """
        return SandpileSortConfig(self.sandpile_struct, {v:self.sandpile_struct.out_degree(v)-1 for v in self.vertices}, self.perm_group)
    
    
    def nonsink_vertices(self):                         ## Returns non-sink vertices
        r"""
            Returns the non-sink vertices of the sorted sandpile.
        """
        return self.sandpile_struct.nonsink_vertices()
    

    def sink(self):                                     ## Returns sink
        r"""
            Returns the sink of the sorted sandpile.
        """
        return self.sandpile_struct.sink()
    

    def sandpile(self):                                 ## Returns the unsorted sandpile
        r"""
            Returns the sandpile without sorting.
        """
        return self.sandpile_struct

    
    def reduced_laplacian(self):                        ## Reduced Laplacian
        r"""
            Returns the reduced Laplacian matrix of the sandpile.
        """
        return self.sandpile_struct.reduced_laplacian()


    def simple_recurrents(self, verbose = True):                            ## Computes the simple recurrents
        r"""
            Returns the list of recurrent configurations ignoring the action of perm_group.

            It's a call of the recurrents() function from Sandpile class.
        """
        return self.sandpile_struct.recurrents(verbose = verbose)
    

    def sorted_recurrents(self, option = 2):                                ## Computes a list of all sorted recurrent configurations
        r"""
            Computes the sorted recurrent configurations.

            - option        : based on the value the search algorithm changes
                = 0     : calls the recurrent() function from Sandpile class [default].
                = 1     : is a modified version of recurrent() function that ignores the same orbits all together.
                = 2     : modified version of the previous, where the duplicate search is more efficient since we sort by configuration degree.
        """
        if option == 0:                                                         ## OPTION 0: computes all recurrents and then quotient by perm_group action
            simpl_rec = self.simple_recurrents()        # Compute all the recurrent configurations
            for i in range(len(simpl_rec)):             # Reorder elements in each configuration
                temp = SandpileSortConfig(self.sandpile_struct, simpl_rec[i], self.perm_group)
                X = temp.sort()
                simpl_rec[i] = [X[j] for j in list(X)]  # Change dictionary in list
            simpl_rec.sort()                            # Remove duplicates
            simpl_rec = list(simpl_rec for simpl_rec,_ in itertools.groupby(simpl_rec))
            # Now back to a dictionary...
            self.sorted_rec = []
            for i in range(len(simpl_rec)):
                sort_dict = {self.vertices[j]:simpl_rec[i][j] for j in range(len(simpl_rec[i]))}
                self.sorted_rec = self.sorted_rec + [sort_dict]
            return self.sorted_rec

        elif option == 1:                                                         ## OPTION 1: modified version of 0, doesn't append the equivalent configurations
            sorted_temp = []                    # Empty sorted_rec list         
            active = [self._max_stable()]       # Start just with the maximal stable
            while active:                       # While the active list is non-empty...
                c = active.pop()
                sorted_temp.append(c)
                for v in self.vertices:
                    cnext = deepcopy(c) # type: ignore                   # Deepcopy the configuration
                    cnext.sandpile_config[v] += 1                        # Add 1 to a vertex in the configuration
                    cnext.sandpile_config = ~cnext.sandpile_config       # Stabilize the new configuration
                    cnext.sort()
                    if (cnext not in active) and (cnext not in sorted_temp) and (cnext != c):            # If it is still to be discovered and not repeating...
                        active.insert(0,cnext)                           # Instead of appending, put at the start of list!
            # Now convert all SandpileSortConfig to dictionaries...
            self.sorted_rec = [x.sandpile_config for x in sorted_temp]
            return self.sorted_rec
        
        elif option == 2:                                                         ## OPTION 2: modified version of 1 that sorts the recurrent configurations and active by degree (invariant for sorting).
                                                                                  #            The idea is to make the search in active and sorted_temp more efficient.
            max_deg = self._max_stable().deg()
            sorted_temp = []                                                                    # Empty sorted_rec list     
            sorted_dict = {i:[] for i in range(max_deg+1)}                                      # Empty sorted_rec dictionary, keyed by degree   
            active = [self._max_stable()]                                                       # Empty active list
            active_dict = {**{max_deg:[self._max_stable()]},**{i:[] for i in range(max_deg)}}       # Empty active dictionary, keyed by degree
            while active:                       # While the active list is non-empty...
                c = active.pop()
                deg_c = c.deg()
                active_dict[deg_c].pop()
                sorted_temp.append(c)               # Append to sorted list
                sorted_dict[deg_c].insert(0,c)      # Append to sorted list dictionary
                for v in self.vertices:
                    cnext = deepcopy(c) # type: ignore                   # Deepcopy the configuration
                    cnext.sandpile_config[v] += 1                        # Add 1 to a vertex in the configuration
                    cnext.sandpile_config = ~cnext.sandpile_config       # Stabilize the new configuration
                    cnext.sort()
                    deg_cnext = cnext.deg()
                    if (cnext not in active_dict[deg_cnext]) and (cnext not in sorted_dict[deg_cnext]) and (cnext != c):            # If it is still to be discovered and not repeating...
                        active.insert(0,cnext)
                        active_dict[deg_cnext].insert(0,cnext)
            # Now convert all SandpileSortConfig to dictionaries...
            self.sorted_rec = [x.sandpile_config for x in sorted_temp]
            return self.sorted_rec

        else:
            raise Exception("The given option is not valid.")
    

    def q_Polynomial(self, opt = 2, override = False):                      ## Computes the q,t polynomial on (level, delay)
        r"""
            Returns the q - polynomial corresponding to the sorted sandpile's recurrent configurations.

            - order     : if specified, it fixes the reading order for the delay statistic.
            - opt       : if specified, it uses a different computing algorithm.
            - override  _ if specified, the sorted recurrents are computed even if sorted_rec is non-empty.
        """
        R = FractionField(QQ['q'])      # type: ignore
        q = R.gen()
        poly = 0*q                                  # Define the polynomial as 0

        if self.sorted_rec == [] or override:       # If sorted recurrents still to be computed...
            self.sorted_recurrents(option=opt)          # ...compute them!
        
        for config in self.sorted_rec:      # Compute the polynomial
            sortedconfig = SandpileSortConfig(self.sandpile_struct, config, self.perm_group, sort = False, verts = self.vertices)
            q_exp = sortedconfig.level()
            #print("Configurazione {} con livello {} e delay {}".format(sortedconfig.sandpile_config, q_exp, t_exp))
            poly = poly + (q**q_exp)
        return poly


    def qt_Polynomial(self, ordered = [], opt = 2, override = False):       ## Computes the q,t polynomial on (level, delay)
        r"""
            Returns the q,t - polynomial corresponding to the sorted sandpile's recurrent configurations.

            - order     : if specified, it fixes the reading order for the delay statistic.
            - opt       : if specified, it uses a different computing algorithm.
            - override  : if specified, the sorted recurrents are computed even if sorted_rec is non-empty.
        """
        R = FractionField(QQ['q, t'])      # type: ignore
        q,t = R.gens()
        poly = 0*q*t                            # Define the polynomial as 0

        if self.sorted_rec == [] or override:   # If sorted recurrents still to be computed...
            self.sorted_recurrents(option=opt)        # ...compute them!
        
        # TODO: be sure that the delay doesn't depend on the order in the same orbit...

        if ordered == []:                   # If there is no explicit order check for a specific one
            if self.specific_opt[0] == "clique-indep" or self.specific_opt[0] == "mul-clique-indep":          # The reading order that defines delay...
                ordered = self.specific_opt[2]
            elif self.specific_opt[0] == "gen-clique-indep" or self.specific_opt[0] == "mulgen-clique-indep":
                ordered = self.specific_opt[3]
            
        if self.specific_opt[0] == "mul-clique-indep":              # Compute the polynomial with k_delay
            for config in self.sorted_rec:
                sortedconfig = SandpileSortConfig(self.sandpile_struct, config, self.perm_group, sort = False, verts = self.vertices)
                q_exp = sortedconfig.level()
                t_exp = sortedconfig.k_delay(order = ordered, check_rec=False)
                #print("Configurazione {} con livello {} e delay {}".format(sortedconfig.sandpile_config, q_exp, t_exp))
                poly = poly + (q**q_exp) * (t**t_exp)
        else:                                                       # Compute the polynomial with regular delay
            for config in self.sorted_rec:
                sortedconfig = SandpileSortConfig(self.sandpile_struct, config, self.perm_group, sort = False, verts = self.vertices)
                q_exp = sortedconfig.level()
                t_exp = sortedconfig.k_delay(order = ordered, check_rec=False)
                #print("Configurazione {} con livello {} e delay {}".format(sortedconfig.sandpile_config, q_exp, t_exp))
                poly = poly + (q**q_exp) * (t**t_exp)
        
        return poly
    

    def associated_ring(self, coeff_ring, order = [], homog = False):       ## Compute the associated ring
        r"""
            Compute the ring associated to the sorted sandpile. The possible arguments are:
                - coeff_ring    : specify the coefficient ring for the polynomial ring.
                - order         : the vertex order associated to variables x0, ..., xn (optional).
                - homog         : asks whether to construct the homogeneous ideal or not (optional, default is False).

            The function returns a list with the quotient ring and a tuple with the polynomial ring and the ideal.
        """
        if not homog:                                           ## Not homogeneous
            # Define the multivariate polynomial ring
            R = PolynomialRing(coeff_ring, ['x%s'%v for v in range(len(self.vertices))])    #type: ignore
            t = R.gens()
            if order == []:
                x = {self.vertices[i]:t[i] for i in range(len(self.vertices))}      # Dictionary with variables
            else:
                x = {order[i]:t[i] for i in range(len(self.vertices))}

            # Create a list with generators for the ideal
            ideal_gens = []
            for v in self.vertices:         # Toppling polynomials
                poly = - x[v]**(self.sandpile_struct.out_degree(v)) + R.prod([x[e[1]] for e in self.sandpile_struct.edges(v, labels = False) if e[1] in self.vertices])
                ideal_gens.append(poly)
            I = R.ideal(ideal_gens)         # Define the ideal

            return (R, I)
        else:                                                   ## Homogeneous
            # Define the multivariate polynomial ring
            R = PolynomialRing(coeff_ring, ['x%s'%v for v in range(len(self.vertices)+1)])    #type: ignore
            t = R.gens()
            if order == []:
                order = self.vertices + [self.sink()]
            x = {order[i]:t[i] for i in range(len(self.vertices)+1)}

            # Create a list with generators for the ideal
            ideal_gens = [x[self.sink()]-1]
            for v in self.vertices:         # Toppling polynomials
                poly = - x[v]**(self.sandpile_struct.out_degree(v)) + R.prod([x[e[1]] for e in self.sandpile_struct.edges(v, labels = False)])
                ideal_gens.append(poly)
            I = R.ideal(ideal_gens)         # Define the ideal

            return (R, I)


    def symm_poly(self, coeff_ring, config, order = []):                    ## Given a configuration, this function returns the symmetrized
        r"""
            This function computes the polynomial symmetric with respect to perm_group associated to the configuration config.
        """
        (R,I) = self.associated_ring(coeff_ring, order = order)    # Define the ring
        t = R.gens()
        if order == []:
            order = self.vertices
        x = {order[i]:t[i] for i in range(len(self.vertices))}

        poly = R.prod([x[v]**config[v] for v in self.vertices])
        Sn = SymmetricGroup(range(len(order)))      #type:ignore
        for subgr in self.perm_group:
            new_poly = 0
            indexes = [i for i in range(len(order)) if order[i] in subgr]
            for sigma in Permutations(indexes):     #type: ignore
                ext_dict = {**{v:v for v in order if v not in subgr},**{subgr[i]:order[sigma[i]] for i in range(len(indexes))}}
                ext_perm = [ext_dict[v] for v in order]
                ext_sigma = Sn([ext_perm.index(v) for v in order])   #type: ignore
                new_poly = new_poly + poly(*ext_sigma(t))
            poly = new_poly
        return poly
    

    def sortrec_ideal(self, coeff_ring, sorted = True, opt = 2, override = False, order = [], homog = False):     ## Compute the sorted recurrent ideal
        r"""
            Compute the ideal to sorted recurrents. The possible arguments are:
                - coeff_ring    : specify the coefficient ring for the polynomial ring.
                - sorted        : boolean, it indicates if the considered sandpile is sorted (optional, default is True).
                - opt           : option for computing sorted recurrents (optional).
                - override      : boolean, if true the sorted recurrents are computed even if sorted_rec is non-empty (optional, default is False)
                - order         : the vertex order associated to variables x0, ..., xn (optional).
                - homog         : asks whether to construct the homogeneous ideal or not (optional, default is False).

            The function returns a list with the quotient ring, the polynomial ring and the ideal.
        """
        if not homog:
            if self.sorted_rec == [] or override:                       # Computes the sorted recurrents if necessary
                self.sorted_recurrents(option = opt)

            (R,I) = self.associated_ring(coeff_ring, sorted=sorted, order=order)     # Construct the associated ring
            t = R.gens()
            if order == []:
                order = self.vertices
            x = {order[i]:t[i] for i in range(len(self.vertices))}
            
            ideal_gens = []                                             # Compute ideal generators for R
            for conf in self.sorted_rec:
                if sorted:
                    poly = self.symm_poly(coeff_ring, conf, order=order)
                else:
                    poly = R.prod([x[v]**conf[v] for v in self.vertices])
                ideal_gens.append(poly)
            J = R.ideal(ideal_gens)

            return (R,I,J)
        else:
            if self.sorted_rec == [] or override:                       # Computes the sorted recurrents if necessary
                self.sorted_recurrents(option = opt)

            (R,I) = self.associated_ring(coeff_ring, sorted=sorted, order=order, homog=True)     # Construct the associated ring
            t = R.gens()
            if order == []:
                order = self.vertices + [self.sink()]
            x = {order[i]:t[i] for i in range(len(self.vertices)+1)}

            ideal_gens = []                                             # Compute ideal generators for R
            for conf in self.sorted_rec:
                if sorted:
                    poly = self.symm_poly(coeff_ring, conf, order=[v for v in order if v != self.sink()])
                else:
                    poly = R.prod([x[v]**conf[v] for v in self.vertices])
                ideal_gens.append(poly)
            J = R.ideal(ideal_gens)

            return (R,I,J)
    
    
    def show(self, default = False):                                        ## Function that displays the Sorted Sandpile
        r"""
            This function plots the sorted sandpile. The possible arguments are:
            - basic = False : if True, the representation is the same as Sandpile.show().

            If specific_opt is non-empty, the show function uses specific parameters:
            - default                   : call to the Sandpile.show() function
            - clique-independent graph  : call to the Sandpile.show() function fixing the position of the vertices,
                                          the sink in the center and the nonsink in a circle.
        """
        if self.specific_opt == [] or default:                         # Default
            if self.sandpile_struct.has_multiple_edges():
                self.sandpile_struct.show(edge_labels=True)
            else:
                self.sandpile_struct.show()
        elif self.specific_opt[0] == "clique-indep":        # Clique-independent graph
            [mu, nu] = self.specific_opt[1]
            mu_num = sum(mu)
            nu_num = sum(nu)
            # Define the position of each vertex
            positions = {**{0:(0,0)},** {i+1:(-np.sin(2*np.pi*i/(mu_num + nu_num)), np.cos(2*np.pi*i/(mu_num + nu_num)))  for i in range(mu_num + nu_num)}}
            # Define the color of each part
            palette = rainbow(len(mu) + len(nu) + 1) # type: ignore
            col = {**{palette[0]:[0]},**{palette[i+1]:self.perm_group[i] for i in range(len(nu) + len(mu))}}

            G = Graph(self.sandpile_struct.dict()).to_undirected() # type: ignore
            
            G.show(pos = positions, vertex_colors = col)
        elif self.specific_opt[0] == "gen-clique-indep":        # Generalized clique-independent graph
            vertex_set = self.specific_opt[1].vertices()
            cliq = []
            indep = []
            for vert in vertex_set:
                if self.specific_opt[2][vert] > 0:
                    cliq.append(vert)
                else:
                    indep.append(vert)
            palette = rainbow(2)    #type: ignore
            col = {palette[0]:cliq, palette[1]:indep}    #type: ignore
            self.specific_opt[1].show(vertex_colors = col, vertex_labels=self.specific_opt[2])
        elif self.specific_opt[0] == "mul-clique-indep":            # Multi clique-independent graph
            [mu, nu] = self.specific_opt[1]
            mu_num = sum(mu)
            nu_num = sum(nu)
            # Define the position of each vertex
            positions = {**{0:(0,0)},**{i+1:(-np.sin(2*np.pi*i/(mu_num + nu_num)), np.cos(2*np.pi*i/(mu_num + nu_num)))  for i in range(mu_num + nu_num)}}
            # Define the color of each part
            palette = rainbow(len(mu) + len(nu) + 1) # type: ignore
            col = {**{palette[0]:[0]},**{palette[i+1]:self.perm_group[i] for i in range(len(nu) + len(mu))}}

            G = Graph(self.sandpile_struct.dict()).to_undirected() # type: ignore
            
            G.show(pos = positions, vertex_colors = col, edge_labels = True)
        elif self.specific_opt[0] == "mulgen-clique-indep":    # Multi generalized clique-independent graph
            vertex_set = self.specific_opt[1].vertices()
            cliq = []
            indep = []
            for vert in vertex_set:
                if self.specific_opt[2][vert] > 0:
                    cliq.append(vert)
                else:
                    indep.append(vert)
            palette = rainbow(2)    #type: ignore
            col = {palette[0]:cliq, palette[1]:indep}    #type: ignore
            self.specific_opt[1].show(vertex_colors = col, vertex_labels=self.specific_opt[2], edge_labels=False)
    

    def export(self, saveopt = 0, opt = 2):                 ## Export informations for the Sorted Sandpile
        r"""
            This function returns the critical information of the sorted sandpile in a format that can be saved using pickle.
            If saveopt = 0, the function returns a list with:
                -   saveopt
                -   a dictionary of the underlying graph
                -   the sink of the sandpile
                -   the permutation group
                -   the specific options for the sandpile
                -   the order of vertices for...
                -   ...the list of sorted recurrents
                -   the qt-polynomial associated to the sorted sandpile.
        """
        qt_poly = self.qt_Polynomial(opt = opt)
        key_list = list(self.sorted_rec[0].keys())
        conflist = [[conf[i] for i in key_list] for conf in self.sorted_rec]
        return [saveopt, self.sandpile_struct.dict(), self.sandpile_struct.sink(), self.perm_group, self.specific_opt, key_list, conflist, qt_poly]


    def save(self, namefile, saveopt = 0, opt = 2):         ## Save the information on a file
        r"""
            This function saves the critical information of the sorted sandpile in a namefile
        """
        info = self.export(saveopt = saveopt, opt = opt)
        
        with open(namefile, 'wb') as handle:
            pickle.dump(info, handle)
    

    def load(namefile):                                     ## Load the information for a Sandpile
        r"""
            This function reads a file and returns a sorted sandpile with the information.
        """
        with open(namefile, 'rb') as handle:
            info = pickle.load(handle)

        if info[0] == 0:
            sorted_rec = []
            keys = info[5]
            for conf in info[6]:
                sort_conf = {keys[i]:conf[i] for i in range(len(keys))}
                sorted_rec.append(sort_conf)
            S = SortedSandpile(info[1], info[2], info[3], opt=info[4], sort_rec = sorted_rec)
            return S
        else:
            raise ImportError("The file cannot be read.")


                    ########################################################
                    #                   Specific Sandpiles                 #
                    ########################################################


def CliqueIndependent_SortedSandpile(mu, nu):                                                                                           ## Specific type of Sandpile
    r"""
        Construction of a Sorted Sandpile on the clique-independent graph given by parameters:
            - mu    : partition associated to the number and size of cliques in the graph.
            - nu    : partition associated to the number and size of independents in the graph.
    """
    mu_num = sum(mu)                                    # Number of independent vertices
    nu_num = sum(nu)                                    # Number of vertices in cliques
    d = {0 : [i+1 for i in range(mu_num + nu_num)]}     # Initialize the dictionary that will define the graph, link the sink to each vertex
    perm_group = []                                     # Initialize the permutation group acting on the graph

    part_first = 1                                      # Keeps track of first vertex of current part
    for part_nu in nu:                                                  # Add edges for independent vertices. For each part...
        for i in range(part_nu):                                        # ...for each vertex in the part...
            d[part_first + i] = [vert for vert in range(mu_num + nu_num + 1) if (vert < part_first) or (vert >= part_first + part_nu)]
                                                                        # ...add all edges except for other vertices in part_nu    
        perm_group.append([part_first+j for j in range(part_nu)])       # Add the permutation orbit for the nu_part
        part_first += part_nu
    mu_rev = copy.copy(mu)              # We need the reversed partition mu...
    mu_rev.reverse()
    for part_mu in mu_rev:
        for i in range(part_mu):
            d[part_first + i] = [vert for vert in range(mu_num + nu_num + 1) if vert != part_first + i]
        perm_group.append([part_first+j for j in range(part_mu)])       # Add the permutation orbit for the mu_part
        part_first += part_mu

    ordered = []
    for i in range(len(mu)):
        temp = copy.copy(perm_group[len(perm_group)-i-1])
        temp.sort()
        ordered = ordered + temp
    for j in range(len(nu)):
        temp = copy.copy(perm_group[len(nu)-j-1])
        temp.sort()
        temp.reverse()
        ordered = ordered + temp
    
    G = Graph(d)            # type: ignore      # Define the clique-independent graph
    specif_opt = ["clique-indep", [mu, nu], ordered]
    '''
        Specific Options:
        1-  list of the defining partitions
        2-  reading order
    '''
    S = SortedSandpile(G, 0, perm_group, specif_opt)
    return S


def General_CliqueIndependent_SortedSandpile(cells_graph, card_cell, order_cells = []):                                                 ## Specific type of Sandpile
    r"""
        Construction of a Sorted Sandpile on the generalized clique-independent graph. Arguments are a graph and a dictionary with values for vertices, such that:
            - nodes:    represents a clique or an independent component with |card_cell[node]| vertices.
                        If the sign of card_cell[node] is:
                            - positive:     we have a clique.
                            - negative:     we have an independent.
            - edges:    each edge correspond to all possible edges between the two cells connected.
    """
    cell_list = cells_graph.vertices()
    
    if order_cells == []:
        order_cells = copy.copy(cell_list)
    
    num_vert = sum([abs(card_cell[i]) for i in cell_list])      # Set the total number of vertices for sandpile

    dict_cell = {}                                              # Dictionary "cell -> list vertices"
    index_next_cell = 1
    for cell in cell_list:
        dict_cell.update({cell:[index_next_cell + j for j in range(abs(card_cell[cell]))]})
        index_next_cell += abs(card_cell[cell])

    order = []                                                  # Reading order for qt-Polynomials
    for cell in order_cells:                                    
        order = order + dict_cell[cell]
    
    edges_set = {0:[i+1 for i in range(num_vert)]}              # Set of edges, sink to everyone
    perm_group = []                                             # Set the permutation group

    now = 1
    index_next_cell = 1
    for cell in cell_list:                              # Add the cell
        perm_group.append(dict_cell[cell])

        for vert in range(abs(card_cell[cell])):
            if card_cell[cell] > 0:                         # Add for clique
                next_neigh = [0] + [index_next_cell + j for j in range(abs(card_cell[cell])) if index_next_cell + j != now]
            else:                                           # Add nothing for independent
                next_neigh = [0]
            # Add complete edges with other cells...
            for neigh_cell in cells_graph.neighbors(cell):
                next_neigh = next_neigh + dict_cell[neigh_cell]

            # Add to the dictionary
            edges_set.update({now:next_neigh})
            now += 1

        index_next_cell += abs(card_cell[cell])

    G = Graph(edges_set)    #type: ignore
    spec_opt = ["gen-clique-indep", cells_graph, card_cell, order]
    '''
        Specific Options:
        1-  Cell graph
        2-  Cell cardinality dictionary
        3-  Reading order (TODO)
    '''
    S = SortedSandpile(G, 0, perm_group, spec_opt)
    return S


def Multi_CliqueIndependent_SortedSandpile(mu, nu, kmul, hmul = -1):                                                                    ## Specific type of Sandpile
    r"""
        Construction of the Sorted Sandpile, given two partitions mu, nu, where edges have multeplicity k in each clique and multeplicity h between components.
    """
    if hmul == -1:     # If third argument is not given, assume it is equal to k
        hmul = kmul

    mu_num = sum(mu)                                    # Number of independent vertices
    nu_num = sum(nu)                                    # Number of vertices in cliques
    d = {0 : [i+1 for i in range(mu_num + nu_num)]}     # Initialize the dictionary that will define the graph, link the sink to each vertex
    perm_group = []                                     # Initialize the permutation group acting on the graph

    part_first = 1                                      # Keeps track of first vertex of current part
    for part_nu in nu:                                          # Add edges for independent vertices. For each part...
        for i in range(part_nu):
            indep = [vert for vert in range(mu_num + nu_num + 1) for mult in range(kmul-1) if (part_first <= vert) and (vert < part_first + part_nu) and (vert != part_first + i)]
            others = [vert for vert in range(mu_num + nu_num + 1) for mult in range(hmul) if ((0 < vert) and (vert < part_first)) or (vert >= part_first + part_nu)]
            d[part_first + i] = [0] + indep + others
                                                                        # ...add all edges except for other vertices in part_nu    
        perm_group.append([part_first+j for j in range(part_nu)])       # Add the permutation orbit for the nu_part
        part_first += part_nu
    mu_rev = copy.copy(mu)                                              # We need the reversed partition mu...
    mu_rev.reverse()
    for part_mu in mu_rev:                                      # Add edges for clique sets. For each part...
        for i in range(part_mu):
            clique = [vert for vert in range(mu_num + nu_num + 1) for mult in range(kmul) if (part_first <= vert) and (vert < part_first + part_mu) and (vert != part_first + i)]
            others = [vert for vert in range(mu_num + nu_num + 1) for mult in range(hmul) if ((0 < vert) and (vert < part_first)) or (vert >= part_first + part_mu)]
            d[part_first + i] = [0] + clique + others
        perm_group.append([part_first+j for j in range(part_mu)])       # Add the permutation orbit for the mu_part
        part_first += part_mu

    ordered = []
    for i in range(len(mu)):
        temp = copy.copy(perm_group[len(perm_group)-i-1])
        temp.sort()
        ordered = ordered + temp
    for j in range(len(nu)):
        temp = copy.copy(perm_group[len(nu)-j-1])
        temp.sort()
        ordered = ordered + temp
    
    G = Graph(d)            # type: ignore      # Define the clique-independent graph
    specif_opt = ["mul-clique-indep", [mu, nu],  ordered, kmul, hmul]
    '''
        Specific Options:
        1-  list of the defining partitions
        2-  reading order
    '''
    S = SortedSandpile(G, 0, perm_group, specif_opt)
    return S


def MultiGeneral_CliqueIndependent_SortedSandpile(cells_graph, card_cell, multi_sink = 1, multiedge_cell = {}, order_cells = []):       ## Specific type of Sandpile
    r"""
        Construction of a Sorted Sandpile on the generalized clique-independent graph. Arguments are a graph and a dictionary with values for vertices, such that:
            - nodes:    represents a clique or an independent component with |card_cell[node]| vertices.
                        If the sign of card_cell[node] is:
                            - positive:     we have a clique.
                            - negative:     we have an independent.
            - edges:    each edge correspond to all possible edges between the two cells connected.
        If multiedge_cell is non-empty, each clique will have edges of multeplicity multiedge_cell[node].
        If cells_graph has multiple edges, the edges between components will be multiple.
    """
    cell_list = cells_graph.vertices()
    
    if order_cells == []:
        order_cells = copy.copy(cell_list)
    
    num_vert = sum([abs(card_cell[i]) for i in cell_list])      # Set the total number of vertices for sandpile

    dict_cell = {}                                              # Dictionary "cell -> list vertices"
    index_next_cell = 1
    for cell in cell_list:
        dict_cell.update({cell:[index_next_cell + j for j in range(abs(card_cell[cell]))]})
        index_next_cell += abs(card_cell[cell])
    
    order = []                                                  # Reading order for qt-Polynomials
    for cell in order_cells:                                    
        order = order + dict_cell[cell]

    edges_set = {0:[i+1 for i in range(num_vert) for k in range(multi_sink)]}   # Set of edges, sink to everyone with multeplicity "multi_sink"
    perm_group = []                                                             # Set the permutation group

    now = 1
    index_next_cell = 1
    for cell in cell_list:                              # Add the cell
        perm_group.append(dict_cell[cell])

        for vert in range(abs(card_cell[cell])):
            # Add cell with its own vertices
            if card_cell[cell] > 0:                         # Add for clique
                if cell in multiedge_cell.keys():               # If we have multiple edges...
                    next_neigh = [0 for k in range(multi_sink)] + [index_next_cell + j for j in range(abs(card_cell[cell])) for k in range(multiedge_cell[cell]) if index_next_cell + j != now]
                else:                                           # If not...
                    next_neigh = [0 for k in range(multi_sink)] + [index_next_cell + j for j in range(abs(card_cell[cell])) if index_next_cell + j != now]
            else:                                           # Add nothing for independent
                next_neigh = [0 for k in range(multi_sink)]
            # Add complete with outer edges
            for neigh_cell in cells_graph.neighbors(cell):
                multi = list(cells_graph.edges(labels=False)).count((cell, neigh_cell))
                next_neigh = next_neigh + [x for x in dict_cell[neigh_cell] for k in range(multi)]

            # Add to the dictionary
            edges_set.update({now:next_neigh})
            now += 1

        index_next_cell += abs(card_cell[cell])
    
    # Conversion to sage-math's format
    multi_edges_set = {}
    for v in range(num_vert + 1):
        vertex_dict = {}
        for w in set(edges_set[v]):
            vertex_dict.update({w:list(edges_set[v]).count(w)})
        multi_edges_set.update({v:vertex_dict})
    # Construct the graph
    G = Graph(edges_set)    #type: ignore
    spec_opt = ["mulgen-clique-indep", cells_graph, card_cell, order]
    '''
        Specific Options:
        1-  Cell graph
        2-  Cell cardinality dictionary
        3-  Reading order
    '''
    S = SortedSandpile(G, 0, perm_group, spec_opt)
    return S

In [ ]:
SandpileSortConfig()

In [ ]:
n=3
S=Multi_CliqueIndependent_SortedSandpile([n],[],2)

In [ ]:
S.show()

In [ ]:
### SageMath implementation of Parking Functions!
import copy
import sys

                                ######################
                                #   CLASS PARKFUNC   #
                                ######################

class ParkFunc():

    def __init__(self, n, m, func = [], w_area = [], w_label = [], specific_opt=[], check=False):
        r'''
            Initialization of the class ParkFunc. The arguments are:
            -   n : number of rows of the parking function
            -   m : number of columns of the parking function
            -   func : a list with the images of values 1,...,n
            -   w_area : a list with the direct path information
            -   w_label : a list with the direct label information
            -   specific_opt : specific options for the parking function
            -   check : if True, it check whether the parameters satisfy the condition for being a parking function

            When called, it initialize the following data:
            -   N : number of rows
            -   M : number of columns
            -   path : the path shape of the corresponding labelled Dyck path (counting all cells of the NxM grid to the east)
            -   label : the labels read from south to north
        '''
        self.N = n                          # Number of rows
        self.M = m                          # Number of columns
        self.ratio = n/m                    # Ratio for the diagonal slope

        # Recognize the specific parking function type
        if specific_opt == []:
            if m%n == 0:                # K TYPE
                specific_opt = ['k-TYPE', floor(m/n)]   # type: ignore
        self.specific_opt = specific_opt    # Specific options for the parking function

        # Compute the path and label information
        if func != []:
            self.label = sorted([i+1 for i in range(self.N)], key = lambda k: k + m*func[k-1])
            self.path = [m - func[i-1] + 1 for i in self.label]
        else:
            self.label = w_label
            self.path = w_area

        if check:           # Check if the parking condition is satisfied
            for i in range(n):
                if w_area[i] < ceil(m - self.ratio*i):          # type: ignore
                    raise ValueError("The path does not satisfy parking function condition...")
    
    def __main_diag__(self):
        r'''
            Returns the main diagonal shape in the right format.
        '''
        return [ceil(self.M - i/self.ratio) for i in range(self.N)]  # type: ignore

    def __eq__(self,other):
        r'''
            Redefines equality between parking functions not to look at metadata.
        '''
        if self.path == other.path and self.label == other.label and self.N == other.N and self.M == other.M:
            return True
        else:
            return False

    def label_word(self):
        return self.label

    def area_word(self):
        diag = self.__main_diag__()
        return [self.path[i] - diag[i] for i in range(self.N)]

    def area(self):
        r'''
            Compute the area statistic.
        '''
        diag = self.__main_diag__()
        return sum([self.path[i] - diag[i] for i in range(self.N)])
    
    def pdinv(self):                        # PDINV: computes pathdinv statistic
        r'''
            Compute the path diagonal inversion statistic.
        '''
        if self.specific_opt[0] != 'k-TYPE':
            raise TypeError('The algorithm does not work with this shape...')
        k = self.specific_opt[1]
        dinv = 0
        # Read all cells above path
        for i in range(self.N):                  # Loop on rows
            for j in range(self.N*k-self.path[i]):       # Loop on columns
                arm = (self.N*k - self.path[i]) - (j+1)
                leg = i - max([h for h in range(self.N) if self.N*k-self.path[h]-1 < j]) - 1
                if (arm <= k*(leg+1)) and (k*leg < arm + 1):
                    dinv += 1
        return dinv

    def tdinv(self):                        # TDINV: computes tdinv statistic
        if self.specific_opt[0] != 'k-TYPE':
            raise TypeError('The algorithm does not work with this shape...')
        k = self.specific_opt[1]

        tdinv = 0
        row_cresc = [self.label.index(h+1) for h in range(self.N)]
        for i in range(self.N):
            rank_c = self.N*self.path[row_cresc[i]] + (self.N*k+1)*row_cresc[i]
            for j in range(self.N-i-1):
                rank_d = self.N*self.path[row_cresc[i+j+1]] + (self.N*k+1)*row_cresc[i+j+1]
                if rank_c < rank_d and rank_d < rank_c + (self.N*k):
                    tdinv += 1
        return tdinv

    def w_maxtdinv(self):                   # W_MAXTDINV: computes the labelling giving the max tdinv
        if self.specific_opt[0] != 'k-TYPE':
            raise TypeError('The algorithm does not work with this shape...')
        k = self.specific_opt[1]
        
        rank_list = [(self.N*k+1)*i + self.N*self.path[i] for i in range(self.N)]
        order = sorted(range(len(rank_list)), key=lambda k: rank_list[k])
        park = sorted(range(len(order)), key=lambda k: order[k])
        return [i+1 for i in park]

    def maxtdinv(self):                     # MAXTDINV: computes maxtdinv statistic
        max_pf = ParkFunc(self.N, self.M, w_area=self.path, w_label=self.w_maxtdinv(), specific_opt=self.specific_opt)
        return max_pf.tdinv()

    def dinv(self):                         # DINV: computes the dinv of a parking function
        #print("PDINV: {}\t TDINV: {}\t MAXTDINV: {}".format(self.pdinv(),self.tdinv(),self.maxtdinv()))
        return self.pdinv() + self.tdinv() - self.maxtdinv()

    def pmaj(self, infos=False):            # PMAJ: computes the pmak of a parking function
        if self.specific_opt[0] != 'k-TYPE':
            raise TypeError('The algorithm does not work with this shape...')
        k = self.specific_opt[1]
        
        pmaj = 0
        plus = 0
        current = self.N+1
        reading_word = []
        contributes = []
        buffer = []
        for i in range(self.N*k):
            # Adding to buffer new labels with k multeplicity
            new = [self.label[j] for j in range(self.N) if self.path[j] == self.N*k-i]
            buffer = buffer + [new[floor(j/k)] for j in range(len(new)*k)]  # type: ignore
            # Get next step
            if current <= min(buffer):          # Starting new ascending chain
                plus += 1
                current = max(buffer)
            else:                               # Continuing current ascending chain
                current = max([w for w in buffer if w < current])
            
            if current not in reading_word:     # Adding pmaj if needed...
                pmaj += plus
                contributes = contributes + [[current,plus]]
            buffer.remove(current)                 # Removing one copy of current from buffer

            # writing down the new letter in reading word
            reading_word = reading_word + [current]
        if infos:
            return [pmaj,contributes,reading_word]
        else:
            return pmaj

    def path_word(self):
        if self.specific_opt[0] != 'k-TYPE':
            raise TypeError('The algorithm does not work with this shape...')
        k = self.specific_opt[1]
        
        rank_list = [(self.N*k+1)*i + self.N*self.path[i] for i in range(self.N)]
        path_order = sorted(range(len(rank_list)), key=lambda k: rank_list[k])      # Reading order of the columns
        path_word = [self.label[path_order[i]] for i in range(self.N)]
        return path_word

    def to_area_pmaj(self,infos = False):   # TO_AREA_PMAJ: this is the Generalized Loehr-Remmel map
        r'''
            This function returns the image of the parking function through the generalized Loehr-Remmel map.
        '''
        if self.specific_opt[0] != 'k-TYPE':
            raise TypeError('The algorithm does not work with this shape...')
        k = self.specific_opt[1]
        
        ### Create the path word, ordered by rank
        rank_list = [(self.N*k+1)*i + self.N*self.path[i] for i in range(self.N)]
        path_order = sorted(range(len(rank_list)), key=lambda k: rank_list[k])      # Reading order of the columns
        path_word = [self.label[path_order[i]] for i in range(self.N)]                      # Labels in order
        #print("The path_word: {}".format(path_word))

        ### Compute the not_tdinv
        # Compute all pairs of cars making tdinv
        tdinv_pairs = []            
        row_cresc = [self.label.index(h+1) for h in range(self.N)]
        for i in range(self.N):
            rank_c = self.N*self.path[row_cresc[i]] + (self.N*k+1)*row_cresc[i]
            for j in range(self.N-i-1):
                rank_d = self.N*self.path[row_cresc[i+j+1]] + (self.N*k+1)*row_cresc[i+j+1]
                if rank_c < rank_d and rank_d < rank_c + (self.N*k+1):
                    tdinv_pairs = tdinv_pairs + [(i+1,i+j+2)]

        # Compute not_tdinv with previous cars
        not_tdinv = []
        for i in range(self.N):
            temp = 0
            for j in range(i):
                if ((path_word[i],path_word[j]) not in tdinv_pairs) and ((path_word[j],path_word[i]) not in tdinv_pairs):
                    temp += 1
            not_tdinv = not_tdinv + [temp]
        #print("The not_tdinv: {}".format(not_tdinv))

        ### Compute the not_dinvcorr
        # Compute all pairs of cars making dinvcorr
        dinvcorr_pairs = []
        for i in range(self.N):
            rank_c = self.N*self.path[i] + (self.N*k+1)*i
            for j in range(self.N-i-1):
                rank_d = self.N*self.path[i+j+1] + (self.N*k+1)*(i+j+1)
                for shift in range(k-1):
                    if rank_d < rank_c + self.N*(k-1) - self.N*shift and rank_c - self.N - self.N*shift < rank_d:
                        dinvcorr_pairs = dinvcorr_pairs + [(self.label[i],self.label[i+j+1])]
        # Compute not_dinvcorr with previous cars
        not_dinvcorr = []
        for i in range(self.N):
            temp = 0
            for j in range(i):
                temp += k - 1 - len([1 for (a,b) in dinvcorr_pairs if (a,b)==(path_word[i],path_word[j]) or (a,b)==(path_word[j],path_word[i])])
            not_dinvcorr = not_dinvcorr + [temp]

        ### Compute the not_dinv
        not_dinv = [not_tdinv[i] + not_dinvcorr[i] for i in range(self.N)]
        #print("The not_dinv: {}".format(not_dinv))

        ### Compute the w_path of the image
        not_dinv_sort = sorted(not_dinv, key=lambda k: k)
        area_word = [self.N*k-not_dinv_sort[i] for i in range(self.N)]
        
        ### Compute the w_label of the image
        label_word = [path_word[i] for i in sorted(range(self.N), key=lambda k: not_dinv[k]*self.N + path_word[k])]
        if not infos:
            return ParkFunc(self.N, self.M, w_area=area_word, w_label=label_word, specific_opt=self.specific_opt)
        else:
            return [ParkFunc(self.N, self.M, w_area=area_word, w_label=label_word, specific_opt=self.specific_opt), not_tdinv, not_dinvcorr]

    def to_dinv_area(self,infos = False):   # TO_DINV_AREA: this is the Generalized Loehr-Remmel inverse
        r'''
            This function returns the image through the generalized Loehr-Remmel inverse.
        '''
        if self.specific_opt[0] != 'k-TYPE':
            raise TypeError('The algorithm does not work with this shape...')
        k = self.specific_opt[1]

        #self.draw()

        ### Compute the pmaj contributes of each label
        [pmaj, contributes, reading_word] = self.pmaj(infos = True)
        lab_ord = [v[0] for v in contributes]       # Ordered list of the inserted labels
        area = [v[1] for v in contributes]          # Ordered list of final area contributes of each label
        #print("Label order {}".format(lab_ord))
        #print("Area {}".format(area))
        ### Compute the area contributes of each label
        #print("Path {}".format(self.path))
        codinv_dict = {self.label[i]:(self.M - self.path[i]) for i in range(self.N)}
        dinv = [k*i - codinv_dict[lab_ord[i]] for i in range(self.N)]
        #print("Dinv {}".format(dinv))

        ### Build up the path step by step
        # First step
        m = 1
        label = [lab_ord[0]]
        path = [k]
        # Consecutive steps
        for j in range(self.N-1):
            if infos:
                print("\nDIMENSION {} PARTIAL PATH {} PARTIAL LABEL {}".format(j,path,label))
            j += 1
            check = False
            #print([(m-(pos+1))*k + area[j] for pos in range(m-1)])
            #tries = [pos+1 for pos in range(m-1) if ((m+1)*k >= ((m+1)-(pos+1))*k + area[j]) and (path[pos]+k >= ((m+1)-(pos+1))*k + area[j]) and (path[pos+1] <= ((m+1)-(pos+1))*k + area[j])] + [m]
            tries = [pos+1 for pos in range(m-1) if ((pos+1)*k >= area[j]) and (path[pos]+k >= (m-pos)*k + area[j]) and (path[pos+1] <= (m-pos)*k + area[j])] + [m]
            if area[j] == 0:
                tries = [0] + tries
            ind = 0
            if infos:
                print("POSITIONS TO TRY {}".format(tries))
            while ind < len(tries) and check == False:
                temp_path1 = [path[i]+k for i in range(tries[ind])]                     # Before new step
                temp_path3 = [path[tries[ind]+i] for i in range(m-tries[ind])]        # After new step
                #print([tries[ind]+i for i in range(m-tries[ind])])        # After new step
                temp_path = temp_path1 + [(m+1-tries[ind])*k + area[j]] + temp_path3    # Provisional path
                
                temp_label1 = [label[i] for i in range(tries[ind])]                     # Before new label
                temp_label3 = [label[tries[ind]+i] for i in range(m-tries[ind])]      # After new label
                temp_label = temp_label1 + [lab_ord[j]] + temp_label3                   # Provisional label
                order = sorted(range(m+1), key = lambda i: temp_label[i])
                norm_label = sorted(range(m+1), key = lambda i: order[i])
                norm_label = [v+1 for v in norm_label]                                  # Normalized label
                #print("Temp {} \t Norm {}".format(temp_label,norm_label))
                if infos:
                    print("First part {}".format(temp_path1))
                    print("Second part {}".format((m+1-tries[ind])*k + area[j]))
                    print("Third part {}".format(temp_path3))
                #print("Trying label {}\t and path {}".format(norm_label,temp_path))
                temp = ParkFunc(m+1,k*(m+1), w_area=temp_path,w_label=norm_label)
                if infos:
                    temp.draw()
                dinv_val = temp.dinv()
                if sum([dinv[i] for i in range(m+1)]) == dinv_val:
                    #print("\tGOOD: We have dinv {}\t with dinv_val {}\t and sum {}".format(dinv, dinv_val,sum([dinv[i] for i in range(m+1)])))
                    label = temp_label
                    path = temp_path
                    check = True
                #else:
                    #print("\tBAD: We have dinv {}\t with dinv_val {}\t and sum {}".format(dinv, dinv_val,sum([dinv[i] for i in range(m+1)])))
                ind += 1
            
            if check == False:
                print("First part {}".format(temp_path1))
                print("Second part {}".format((m+1-tries[ind-1])*k + area[j]))
                print("Third part {}".format(temp_path3))
                raise ValueError("PROBLEMO!")
            m+=1
        
        return ParkFunc(self.N, self.M, w_area=path, w_label=label)


                                ##########################
                                #   PARKFUNC UTILITIES   #
                                ##########################

    def draw(self, stats=True):
        r'''
            Function that draws the path
        '''
        #print('Path {} and {}'.format(self.path,self.M))
        diag = self.__main_diag__()
        for i in range(self.N):
            i = self.N - i - 1
            row1 = '   ' * (self.M - self.path[i])
            row2 = ' {:>2}'.format(self.label[i])
            row3 = '|##' * (self.path[i] - diag[i])
            row4 = '|  ' * (diag[i]) + '|'

            buff1 = '   ' * (self.M - self.path[i]+1)
            buff2 = '+--' * (self.path[i]) + '+'
            print('\t' + buff1 + buff2)
            print('\t' + row1 + row2 + row3 + row4)
        print('\t' + '   ' + '+--' * self.M + '+')
        if stats:
            print(" Label word:\t{}".format(self.label_word()))
            print(" Area word:\t{}".format(self.area_word()))
            print(" Area: {}\t Dinv: {}\t Pmaj: {}".format(self.area(),self.dinv(),self.pmaj()))

def kDyckPaths(n,k):                            # kDYCKPATHS: computes the set of all (nk,n)-Dyck paths
    possible_paths = [[1]]
    for i in range(n*(k+1)-1):
        new_possible = []
        for part_path in possible_paths:
            height = sum(part_path)
            distance = sum([1-part_path[j] for j in range(i+1)])
            if height == n:                 # Reached max height, just 0's
                temp = copy.deepcopy(part_path)
                temp.append(0)
                new_possible.append(temp)
            elif k*height < distance + 1:   # Reached the "diagonal", just a 1
                temp = copy.deepcopy(part_path)
                temp.append(1)
                new_possible.append(temp)
            else:
                temp = copy.deepcopy(part_path)
                temp.append(0)
                new_possible.append(temp)
                temp = copy.deepcopy(part_path)
                temp.append(1)
                new_possible.append(temp)
        possible_paths = new_possible
    dyck_k_paths = [[sum([1-w[i+j] for j in range(len(w)-i)]) for i in range(len(w)) if w[i]==1] for w in possible_paths]
    return dyck_k_paths

def nkParkingFunctions(n,k, display=False):     # kPARKINGFUNCTIONS: computes the set of all (n,k)-parking functions
    parking_functions = []
    counter = 0
    # Compute total number of Dyck paths
    totalnum = binomial(n*(k+1)+1, n)/(n*(k+1)+1)           # type: ignore
    print("Starting the algorithm to compute all parking functions...")
    # Compute all possible set partitions
    set_partitions = {}
    for lamb in Partitions(n):                              # type: ignore
        temp = []
        for possible in OrderedSetPartitions(n,Composition(lamb)):       # type: ignore
            poss = [list(part) for part in possible]
            temp.append(poss)
        lambt = tuple(lamb)
        set_partitions.update({lambt:temp})

    ### Loop on k-Dyck paths
    for path in kDyckPaths(n,k):
        if display:
            counter += 1
            perc = floor(100*counter/totalnum)              # type: ignore
            sys.stdout.write('\r')                          # Reset to start of line
            sys.stdout.write("Percentage %3d %%, computing parking functions for path %6d of %6d" % (perc, counter, totalnum))
            sys.stdout.flush()

        # Compute the partitions we want
        rec_partition = [len([1 for w in path if w==n*k-i]) for i in range(n*k) if (n*k-i) in path]
        #print(rec_partition)
        ord_partition = copy.copy(rec_partition)
        ord_partition.sort()
        ord_partition.reverse()
        #print(ord_partition)
        ord_partitiont = tuple(ord_partition)
        #print(ord_partitiont)
        part_decr_ord = sorted(range(len(rec_partition)), key=lambda i: -n*rec_partition[i])
        read_order = sorted(range(len(part_decr_ord)), key=lambda i: part_decr_ord[i])
        for set_part in set_partitions[ord_partitiont]:
            label = []
            for j in read_order:
                temp2 = list(list(set_part)[j])
                temp2.sort()
                label = label + temp2
            parking_functions.append(ParkFunc(n,n*k,w_area=path,w_label=label))
    
    print("\n...the algorithm to compute all parking functions has finished.")
    return parking_functions

def qt_Polynomial_dinv_area(set_paths,ring):           # GEN_POLY: polynomial generated by paths
    q = ring.gens()[0]
    t = ring.gens()[1]
    poly = 0*q*t
    for path in set_paths:
        poly = poly + (q**path.dinv())*(t**path.area())
    return poly

def qt_Polynomial_area_pmaj(set_paths,ring):           # GEN_POLY: polynomial generated by paths
    q = ring.gens()[0]
    t = ring.gens()[1]
    poly = 0*q*t
    for path in set_paths:
        poly = poly + (q**path.area())*(t**path.pmaj())
    return poly

def menk(aa,k): #remove k from each element of a list
    return [a-k for a in aa]

def piuk(aa,k): #add k to each element of a list
    return [a+k for a in aa]

from itertools import product
from numpy import argmin

def easy_bounce(path): #calculate bounce stat for Dyck Paths ([1,1,-1,1,-1,-1]) from 0,0 to n,n
    pp=deepcopy(path)
    n=len(pp)
    gg=[sum(pp[:i]) for i in range(n)]
    c=-min(gg)
    j=argmin(gg)
    start=0
    pos=start
    bounce=[]
    i=0
    bb=0
    bs=[]
    i=0
    while i<n:
        if sum(bounce[:i])==0:
            bounce.append(1)
        else:
            if bounce[-1]==-1:
                bounce.append(-1)
            else:
                if sum(bounce[:i])==sum(pp[:i]):
                    if pp[i]==-1:
                        bb+=1
                    bounce.append(pp[i])
                else:
                    bounce.append(1)
        bs.append(bb)
        i+=1
    return sum([bs[i] for i in range(n) if pp[i]==1])

def path_ord(p1,p2): #It is true if path p1 is above path p2 (paths with 1 and -1)
    n=len(p1)
    g1=[sum(p1[:i]) for i in range(n)]
    g2=[sum(p2[:i]) for i in range(n)]
    x=True
    i=0
    while x and i<n:
        if g1[i]<g2[i]:
            x=False
        i+=1
    return x

def my_Paths(n): #list of all paths from 0,0 to n,n (with 1 and -1)
    return [list(seq) for seq in product([-1, 1], repeat=2*n) if sum(seq) == 0]

def my_DyckPaths(n): #list of all Dyck paths from 0,0 to n,n (with 1 and -1)
    dd=[(-1)**i for i in range(2*n)]
    return [pp for pp in my_Paths(n) if path_ord(pp,dd)]

def from_my_to_area(pp): #convert a path with 1 and -1 in its complete! area word
    aa=list(pp)
    n=len(aa)/2
    return [[sum(aa[:i]) for i in range(2*n) if aa[i]==1][j]+n-j for j in range(n)]

def from_area_to_my(pp): #convert a completh area word in a path with 1 and -1
    n=len(pp)
    bb=list(pp)+[0]
    x=[]
    for i in range(n):
        x=x+[1]+[-1 for a in range(bb[i]-bb[i+1])]
    return x

def sc_bo(n,k,dizi=[],perc=[],start=0): 
    #return a list of dictionaries, such that the i-th ([i-1]...) is the list of all paths in n,k*n rectangular
    #above the diagonal, with the bounce decomposition statistic
    if start==0:
        dizi=[{} for a in range(k+1)]    
        dizi[1].update({tuple(menk(from_my_to_area(a),1)):easy_bounce(a) for a in my_DyckPaths(n)})
        perc=[{}]+[{tuple(menk(a,i)):1 for a in kDyckPaths(n,i)} for i in range(1,k+1)]
    if len(dizi[k])>0:
        return [dizi[j] for j in range(1,k+1)]
    km1=sc_bo(n,k-1,dizi,perc,1)[-1]
    for aa in km1:
        for bb in dizi[1]:
            cc=[list(aa)[i]+list(bb)[i] for i in range(n)]
            if tuple(cc) in perc[k]:
                new=easy_bounce(from_area_to_my(list(bb)))+dizi[k-1][aa]
                if tuple(cc) in dizi[k]:
                    if dizi[k][tuple(cc)]>new:
                        dizi[k][tuple(cc)]=new
                else:
                    dizi[k].update({tuple(cc):new})
    return [dizi[j] for j in range(1,k+1)]
        


In [ ]:
n=3
k=3
#3,12  4,8   5,8  6,5
diz=sc_bo(n,k)
print(diz)
for bb in diz[-1]:#check if the bounce decomposition statistic is the same of the new pmaj
    if diz[-1][bb]!=ParkFunc(n,n*k,w_area=piuk(bb,k),w_label=list(range(1,n+1))).pmaj():
        print(bb)
print('ciao')

In [ ]:
for aa in kDyckPaths(n,k):
    print(menk(aa,k))
    print(ParkFunc(n,n*k,w_area=aa,w_label=list(range(1,n+1))).pmaj())
    print()